In [ ]:
!/opt/bin/nvidia-smi

Wed May 11 18:25:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import json
import os
import time
import re
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/NLP_task1/train.csv')
df_dev = pd.read_csv('/content/drive/MyDrive/NLP_task1/dev.csv')
df_test = pd.read_csv('/content/drive/MyDrive/NLP_task1/test.csv')

In [ ]:
df_train.isnull().any()

label            False
text             False
retweet_count    False
dtype: bool

In [ ]:
df_train = df_train.dropna(axis=0, how='any')
df_dev = df_dev.dropna(axis=0, how='any')
# df_test = df_test.dropna(axis=0, how='any')

In [ ]:
df_train.isnull().any()

label            False
text             False
retweet_count    False
dtype: bool

In [ ]:
df_dev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536 entries, 0 to 535
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   label          536 non-null    int64 
 1   text           536 non-null    object
 2   retweet_count  536 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 16.8+ KB


# New Section

In [ ]:
BERT_BASE = 'bert-base-uncased'

In [ ]:
!pip install torch torchvision transformers

from transformers import BertModel

bert_model = BertModel.from_pretrained(BERT_BASE)

print("Done loading BERT model.")    

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Done loading BERT model.


In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer
import pandas as pd
import numpy as np

class rumourset(Dataset):

    def __init__(self,df,having_label,maxlen):

        #Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.texts = df["text"]
        self.having_label = having_label
        self.retweet_count = df["retweet_count"]
        if having_label:
          self.label = df["label"]
        self.maxlen = maxlen

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = str(self.texts[index])

        retweet_count = self.retweet_count[index]

        if self.having_label:
          label = int(self.label[index])


        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.maxlen,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        if self.having_label:  
          return encoding['input_ids'].flatten(), encoding['attention_mask'].flatten(),encoding['token_type_ids'].flatten(),torch.tensor([int(retweet_count)*5]),torch.tensor([label])
        else:
          return encoding['input_ids'].flatten(), encoding['attention_mask'].flatten(),encoding['token_type_ids'].flatten(),torch.tensor([int(retweet_count)*5])


In [ ]:
df_train["retweet_count"]

0         0
1       185
2         1
3       123
4         1
       ... 
1558      1
1559     83
1560    109
1561    177
1562      4
Name: retweet_count, Length: 1563, dtype: int64

In [ ]:
!pip install emoji

In [ ]:
df_merge = pd.concat([df_train,df_dev])


In [ ]:
from torch.utils.data import DataLoader


MAX_LEN = 512
BATCH_SIZE = 8
NUM_WORKERS = 1
SHUFFLE = True

train_set = rumourset(df_train, True,maxlen = MAX_LEN)
train_loader = DataLoader(train_set, batch_size = BATCH_SIZE,shuffle = SHUFFLE, num_workers = NUM_WORKERS)

merge_set = rumourset(df_merge, True,maxlen = MAX_LEN)
merge_loader = DataLoader(merge_set, batch_size = BATCH_SIZE,shuffle = SHUFFLE, num_workers = NUM_WORKERS)


dev_set = rumourset(df_dev, True,maxlen = MAX_LEN)
dev_loader = DataLoader(dev_set, batch_size = BATCH_SIZE,shuffle = SHUFFLE, num_workers = NUM_WORKERS)

    
    
test_set = rumourset(df_test, False,maxlen = MAX_LEN)
test_loader = DataLoader(test_set, batch_size = 1,shuffle = False, num_workers = NUM_WORKERS)


In [ ]:
## build the bert NN model 

import torch
import torch.nn as nn
from transformers import BertModel
import torch.nn as nn
import torch.optim as optim


class RumorClassifier(nn.Module):

    def __init__(self):
        super(RumorClassifier, self).__init__()
        #Instantiating BERT model object 
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = torch.nn.Dropout(0.3)
        self.linear0 = torch.nn.Linear( 769, 256)
        self.linear = torch.nn.Linear( 256, 1)
        



    def forward(self, encode_text, attn_masks,token_type_ids,retweet_count):
        output = self.bert_layer(encode_text, attention_mask = attn_masks, token_type_ids=token_type_ids,return_dict=True)
        output_dropout = self.dropout(output.pooler_output)
        output = torch.tanh(self.linear0(torch.cat((output_dropout,retweet_count),1)))
        output = self.linear(output)
        return output




In [ ]:

gpu = 0 #gpu ID
NNbert = RumorClassifier()
NNbert.cuda(gpu)

## loss function 
criterion = nn.BCEWithLogitsLoss()

## adam optimizer
optimizer = optim.Adam(NNbert.parameters(), lr = 2e-5)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
pip install torchmetrics

In [ ]:
# the structure of the code is modified from Jey Han Lau. 2021. The Project Specification to 352 COMP90042 Natural Language Processing. The 353 University of Melbourne, Carlton VIC, AU.

from torchmetrics.functional import f1_score, recall , precision


import time
def train(nn_model, criterion, optimizer, train_loader,dev_loader, max_eps, gpu):
    best_acc = 0
    best_f1score = 0 
    st = time.time()
    for ep in range(max_eps):
        nn_model.train()
        
        for it, (seq, attn_masks,token_type_ids,retweet_count,labels) in enumerate(train_loader):
            #Clear gradients
            optimizer.zero_grad()  
            #Converting these to cuda tensors
            seq, attn_masks,token_type_ids,retweet_count,labels = seq.cuda(gpu), attn_masks.cuda(gpu),token_type_ids.cuda(gpu),retweet_count.cuda(gpu),labels.cuda(gpu)


            #Obtaining the output from the model
            output = nn_model(seq, attn_masks,token_type_ids,retweet_count)

            #Computing loss
            loss = criterion(output, labels.float())

            #Backpropagating the gradients
            loss.backward()

            #Optimization step
            optimizer.step()
              
            if it % 100 == 0:
                
                acc = get_accuracy(output, labels)
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss.item(), acc, (time.time()-st)))
                st = time.time()

        torch.cuda.empty_cache()
        dev_acc, dev_loss = evaluate(nn_model, criterion,dev_loader,gpu)
        print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, dev_acc,dev_loss))
        if dev_acc > best_acc :
            print("Best development accuracy improved from {} to {}, saving model...".format(best_acc, dev_acc))
            # print("Best development f1score improved from {} to {}, saving model...".format(best_f1score, dev_f1score))
            best_acc = dev_acc
            # best_f1score = dev_f1score
            torch.save(nn_model.state_dict(), 'sstcls_{}.dat'.format(ep))



def get_accuracy(output, labels):
    probs = torch.sigmoid(output)
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs == labels).float().mean()
    return acc




def evaluate(NNbert, criterion, dataloader,gpu):
    NNbert.eval()

    mean_acc, mean_loss = 0, 0
    count = 0


    with torch.no_grad():
        for seq, attn_masks, token_type_ids,retweet_count,labels in dataloader:
            seq, attn_masks,token_type_ids, retweet_count,labels = seq.cuda(gpu), attn_masks.cuda(gpu),token_type_ids.cuda(gpu),retweet_count.cuda(gpu),labels.cuda(gpu)
            output = NNbert(seq, attn_masks,token_type_ids,retweet_count)
            mean_loss += criterion(output, labels.float()).item()

            mean_acc += get_accuracy(output, labels)
            count += 1

  

    return mean_acc / count, mean_loss / count

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  429550 KB |  429550 KB |  429550 KB |       0 B  |\n|       from large pool |  428288 KB |  428288 KB |  428288 KB |       0 B  |\n|       from small pool |    1262 KB |    1262 KB |    1262 KB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |  429550 KB |  429550 KB |  429550 KB |       0 B  |\n|       from large pool |  428288 KB |  428288 KB |

In [ ]:
num_epoch = 6
#fine-tune the model
train(NNbert, criterion, optimizer, train_loader,dev_loader,num_epoch,gpu)

Iteration 0 of epoch 0 complete. Loss: 0.8094308376312256; Accuracy: 0.375; Time taken (s): 1.0006449222564697
Iteration 100 of epoch 0 complete. Loss: 1.0072904825210571; Accuracy: 0.75; Time taken (s): 69.60897970199585
Epoch 0 complete! Development Accuracy: 0.8190298080444336; Development Loss: 0.38568960486063314
Best development accuracy improved from 0 to 0.8190298080444336, saving model...
Iteration 0 of epoch 1 complete. Loss: 0.8320029973983765; Accuracy: 0.875; Time taken (s): 88.15421152114868
Iteration 100 of epoch 1 complete. Loss: 0.3375263214111328; Accuracy: 0.875; Time taken (s): 73.99794578552246
Epoch 1 complete! Development Accuracy: 0.8264924883842468; Development Loss: 0.3493307253985263
Best development accuracy improved from 0.8190298080444336 to 0.8264924883842468, saving model...
Iteration 0 of epoch 2 complete. Loss: 0.42427974939346313; Accuracy: 0.75; Time taken (s): 93.25970792770386
Iteration 100 of epoch 2 complete. Loss: 0.3273555636405945; Accuracy: 0

In [ ]:
def get_predictions(NNbert, test_loader):
    NNbert.eval()
    out_texts = []
    predictions = []

    with torch.no_grad():
      tweet_ids = []
      
      for seq, attn_masks,token_type_ids,retweet_count in test_loader:
          seq, attn_masks,token_type_ids,retweet_count = seq.cuda(gpu), attn_masks.cuda(gpu),token_type_ids.cuda(gpu),retweet_count.cuda(gpu)
          outputs = NNbert(seq,attn_masks,token_type_ids,retweet_count)
          probs = torch.sigmoid(outputs)
          soft_probs = (probs > 0.5).long()
          # tweet_ids.append(tweet_id)
          predictions.append(soft_probs)


    predictions = torch.stack(predictions).cuda(gpu)

    return predictions

In [ ]:
def get_dev_predictions(NNbert, dev_loader):
    NNbert.eval()
    out_texts = []
    predictions_dev = []
    targets =[]

    with torch.no_grad():
      tweet_ids = []
      
      for seq, attn_masks,token_type_ids,retweet_count,labels in dev_loader:
          seq, attn_masks,token_type_ids,retweet_count,labels = seq.cuda(gpu), attn_masks.cuda(gpu),token_type_ids.cuda(gpu),retweet_count.cuda(gpu),labels.cuda(gpu)
          outputs = NNbert(seq,attn_masks,token_type_ids,retweet_count)
          probs = torch.sigmoid(outputs)
          soft_probs = (probs > 0.5).long()
          # tweet_ids.append(tweet_id)
          predictions_dev.append(soft_probs)
          # print(labels)
          targets.append(labels)
          # print(targets)
          

    targets = torch.stack(targets).cuda(gpu)
    predictions_dev = torch.stack(predictions_dev).cuda(gpu)

    return predictions_dev,targets

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
predictions_dev, targets = get_dev_predictions(NNbert, dev_loader)

In [ ]:
pre_dev = predictions_dev.cpu().tolist()
flat_list_pre1 = [item for sublist in pre_dev for item in sublist]
flat_list_pre2 = [item for sublist in flat_list_pre1 for item in sublist]

# print(flat_list_pre2)

In [ ]:
label_dev = targets.cpu().tolist()
flat_list_label1 = [item for sublist in label_dev for item in sublist]
flat_list_label2 = [item for sublist in flat_list_label1 for item in sublist]
# print(flat_list_label2)

In [ ]:
print(classification_report(flat_list_label2,flat_list_pre2,target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.98      0.97      0.97       421
           1       0.89      0.91      0.90       115

    accuracy                           0.96       536
   macro avg       0.93      0.94      0.94       536
weighted avg       0.96      0.96      0.96       536



In [ ]:
predictions = get_predictions(NNbert, test_loader)

def convert_label(num):
    if num == 0:
        return 'nonrumour'
    else:
        return 'rumour'

prediction_list = predictions.tolist()

In [ ]:
output= []
for prediction in prediction_list:
  output.append(prediction[0][0])

In [ ]:
df_result = pd.DataFrame (output, columns = ['predicted'])

In [ ]:
df_result['Id'] = range(0,len(df_result))

In [ ]:
df_result.to_csv('prediction.csv',index = False)

In [ ]:
df_result